In [ ]:
from typing import List
import pickle
from tqdm.auto import tqdm
from utils import CausalShell, Translator, T5Shell, SaigaShell

import pandas as pd

In [ ]:
def get_q_len(qs):
    tqs = []
    for v in qs.values():
        if len(v) > 2:
            tqs += v
    tqs = list(set(tqs))
    return len(tqs)

def a_rate(model, q, a):
    return model.generate(f'''Я задал вопрос: "{q}"

Мне ответили:
"{a}"

Оцени ответ по шкале от 1 до 5. В ответе дай оценку в формате x/5 и короткое обоснование.
''')

def a_rates(model, data):
    data["full rate"] = [a_rate(arm, row["Q"], row["generated_text"]) for i, row in tqdm(data.iterrows(), total=len(data))]
    return data

# Q gen

In [ ]:
qgm = CausalShell("microsoft/Phi-3-mini-4k-instruct")
# translator = T5Shell()

In [ ]:
def q_gen(n=30, to_rus=False):
    qs = []
    n_batch = n // 30 + 1 * ((n % 30) != 0)
    for _ in tqdm(range(n_batch)):
        gqs = qgm.generate("Подготовь 30 различных интересных вопросов для квиза на общие темы не личного характера")
        # if to_rus:
        #     gqs = translator.translate(gqs, "ru")[0]
        qs += gqs.split("\n")
    return qs

def context_q_gen(contexts: List[str]):
    result = {}
    for c in tqdm(contexts):
        gqs = qgm.generate(f"Придумай 20 вопросов по теме {c} без ответов")
        result[c] = [q for q in gqs.split('\n') if "?" in q]
    return result
        

In [ ]:
raa = context_q_gen([
    "алгоритмы",
    "структуры данных",
    "обработка исключений",
    "обработка ошибок",
    "файловая система",
    "операционная система",
    "работа с файлами",
    "типы файлов",
    "кодирование файлов",
    "базы данных",
    "SQL",
    "не реляционные базы данных",
    "распределенные базы данных",
    "большие данные",
    "ИИ",
    "интернет вещей",
    "машинное обучение",
    "обучение без учителя",
    "проклятье размерностей",
    "статистика",
    "линейная алгебра",
    "математический анализ",
    "обучение с подкреплением",
    "не реляционные базы данных",
    "СУБД",
    "антифрод системы",
    "рекомендательные системы",
    "тестирование кода",
    "отладка кода",
    "компьютерная графика",
    "геймдизайн",
    "разработка мобильных приложений",
    "разработка игр"
])
raa

In [ ]:
ra = context_q_gen([
    "большие данные"
])
ra

In [ ]:
r.update(ra)
r

In [ ]:
for k, v in r.items():
    r[k] = list(set(v + raa[k]))

In [ ]:
with open("qs.pickle", "wb") as f:
    pickle.dump(r, f)

# LLM tests

In [ ]:
def split_qs():
    with open("qs.pickle", "rb") as f:
        qs = pickle.load(f)
    
    new_qs = {"R1": {}, "R2": {}}
    for k, v in qs.items():
        edge = len(v)//2
        if edge > 2:
            new_qs["R1"][k] = v[:edge]
            new_qs["R2"][k] = v[edge:]
    return new_qs

def get_answers(qs, model):
    model_name = model.model.metadata["general.name"]
    answers = []
    pbar = tqdm(total=get_q_len(qs))

    c_num = len(qs)
    for c_count, (k, v) in enumerate(qs.items(), start=1):
        pbar.desc = f"{k} {c_count}/{c_num}"
        for q in v:
            tr={"Q": q, "category": k, "model": model_name}
            tr.update(model.generate(f"Ответь на вопрос: {q}"))
            answers.append(tr)
            pbar.update()
        pd.DataFrame(answers).to_csv(f"answers_{model_name}.csv", index=False)
    return answers

In [ ]:
qs = split_qs()
print(get_q_len(qs["R1"]))
print(get_q_len(qs["R2"]))

In [ ]:
qm16 = SaigaShell(r"C:\Users\dmatr\Downloads\model-f16.gguf")

In [ ]:
r = get_answers(qs["R1"], qm16)

In [ ]:
qm8 = SaigaShell(r"C:\Users\dmatr\Downloads\model-q8_0.gguf")

In [ ]:
r = get_answers(qs["R2"], qm8)

# A rate

In [ ]:
arm = CausalShell("microsoft/Phi-3-mini-4k-instruct")

In [ ]:
answers1 = pd.read_csv("R1_answers_saiga_llama3_8b.csv")
answers1 = a_rates(arm, answers1)
answers1.to_csv("R1_rate.csv", index=False)

In [ ]:
answers2 = pd.read_csv("R2_answers_saiga_llama3_8b.csv")
answers2 = a_rates(arm, answers2)
answers2.to_csv("R2_rate.csv", index=False, header=False, mode="a")

In [ ]:
answers1["model"] = "saiga_f16"
answers2["model"] = "saiga_q8"
answers = pd.concat([answers1, answers2], axis=0, ignore_index=True)
answers["rate"] = answers["full rate"].apply(lambda x: x[x.find('/5') - 1])

In [ ]:
answers.to_excel("answers.xlsx", index=False)

# AB test

In [ ]:
# !pip install hypex==1.0.0a0

In [7]:
from hypex.dataset import Dataset, InfoRole, TreatmentRole, TargetRole
from hypex.experiments.ab import ABTest

In [8]:
# answers = pd.read_excel("answers.xlsx")
answers = Dataset(
    roles={
        "model": TreatmentRole(),
        "n_tokens": TargetRole(),
        "time": TargetRole(), 
        "category": TargetRole(str),
        "rate": TargetRole(), 
        "speed": TargetRole(),
    }, data="answers.xlsx",
)
# answers["model"] = answers["model"].replace("saiga_f16", 0).replace("saiga_q8", 1)
answers

                                                     Q          category  \
0    ### Answer:1. Какая разница между пулом процес...  структуры данных   
1    8. Как реализовать кольцо стек, используя связ...  структуры данных   
2    6. Как создать и отсортировать массив с исполь...  структуры данных   
3    4. Какие фаクс данных используются для хранения...  структуры данных   
4    9. Что такое прототипное наследование в JavaSc...  структуры данных   
..                                                 ...               ...   
832  14. Как настроить физическую симуляцию, чтобы ...    разработка игр   
833  15. Как использовать Unity's Terrain для созда...    разработка игр   
834     11. Как создать собственный коллактор в Unity?    разработка игр   
835      **18. How do game engines handle animation?**    разработка игр   
836  **3. What are the main components of a game en...    разработка игр   

         model  n_tokens        time  \
0    saiga_f16       578  252.267314   
1    sa

In [9]:
result = ABTest().execute(answers)

c:\Projects\HypEx\hypex\dataset\backends\pandas_backend.py:305: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groups = self.data.groupby(by, **kwargs)
c:\Projects\HypEx\hypex\dataset\backends\pandas_backend.py:305: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groups = self.data.groupby(by, **kwargs)
c:\Projects\HypEx\hypex\dataset\backends\pandas_backend.py:486: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories 

In [12]:
result.resume.data

,feature,group,TTest pass,TTest p-value,control mean,test mean,difference,difference %
0,n_tokens,saiga_q8,NOT OK,3.195728e-01,466.690998,479.774648,13.083650,2.803493
1,time,saiga_q8,OK,2.089633e-66,188.198086,98.663469,-89.534618,-47.574670
2,rate,saiga_q8,NOT OK,8.973242e-01,3.914842,3.922535,0.007693,0.196518
3,speed,saiga_q8,OK,4.335499e-122,2.670469,5.338672,2.668203,99.915145


In [11]:
result.sizes 

          control size  test size  control size %  test size %     group
saiga_q8           411        426       49.103943    50.896057  saiga_q8